In [1]:
from pymilvus import connections, Collection
import pandas as pd

connections.connect(uri="http://localhost:19530")  # Replace with your Milvus server IP

In [2]:
collection = Collection(name="thu_vien_phap_luat")

## Search


In [3]:
from pymilvus import AnnSearchRequest
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from pymilvus import RRFRanker

bge_m3_ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
rerank = RRFRanker()

c:\Users\daoqu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]
c:\Users\daoqu\AppData\Local\Programs\Python\Python310\lib\site-packages\FlagEmbedding\BGE_M3\modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issu

In [4]:
def query(text, limit=3):
    query_embeds = bge_m3_ef.encode_queries([text])

    title_sparse_search_params = {
        "data": query_embeds["sparse"],  # Query vector
        "anns_field": "title_sparse",  # Vector field name
        "param": {
            "metric_type": "IP",  # This parameter value must be identical to the one used in the collection schema
        },
        "limit": limit,  # Number of search results to return in this AnnSearchRequest
    }
    title_sparse_request = AnnSearchRequest(**title_sparse_search_params)

    title_dense_search_params = {
        "data": query_embeds["dense"],
        "anns_field": "title_dense",
        "param": {
            "metric_type": "COSINE",
            "ef": 20,
        },
        "limit": limit,
    }
    title_dense_request = AnnSearchRequest(**title_dense_search_params)

    content_sparse_search_params = {
        "data": query_embeds["sparse"],
        "anns_field": "content_sparse",
        "param": {
            "metric_type": "IP",
        },
        "limit": limit,
    }
    content_sparse_request = AnnSearchRequest(**content_sparse_search_params)

    content_dense_search_params = {
        "data": query_embeds["dense"],
        "anns_field": "content_dense",
        "param": {
            "metric_type": "COSINE",
            "ef": 20,
        },
        "limit": limit,
    }
    content_dense_request = AnnSearchRequest(**content_dense_search_params)

    reqs = [
        title_sparse_request,
        title_dense_request,
        content_sparse_request,
        content_dense_request,
    ]

    collection.load()
    res = collection.hybrid_search(
        reqs,  # List of AnnSearchRequests created in step 1
        rerank,  # Reranking strategy specified in step 2
        limit=limit,  # Number of final search results to return
        output_fields=["id", "title", "content_text", "url"],
    )
    return res

In [7]:
results = query(
    "Đối với xe cơ giới thuộc đối tượng miễn kiểm định lần đầu, đơn vị đăng kiểm cần làm gì?"
)

# print(results)
for i in results[0]:
    print(i.entity.get("content_text"))

2.
    
     Lập hồ sơ phương tiện đối
với xe cơ giới được miễn kiểm định lần đầu
    
    là việc đơn vị đăng kiểm kiểm
tra, đánh giá hồ sơ phương tiện để lập hồ sơ phương tiện, cấp Giấy chứng nhận
kiểm định, Tem kiểm định đối với xe cơ giới được miễn kiểm định lần đầu theo
quy định tại Thông tư số
    
     16/2021/TT-BGTVT
    
    ngày 12/8/2021 của Bộ trưởng Bộ Giao thông vận tải quy định về kiểm định an
toàn kỹ thuật và bảo vệ môi trường phương tiện giao thông cơ giới đường bộ (được
sửa đổi, bổ sung bởi Thông tư số
    
     02/2023/TT-BGTVT
    
    ngày 21/3/2023, Thông tư số
    
     08/2023/TT-BGTVT
    
    ngày 02/06/2023).
   

    3.
    
     In lại Giấy chứng nhận kiểm
định và Tem kiểm định đối với xe cơ giới
    
    là việc đơn vị đăng kiểm in lại Giấy
chứng nhận kiểm định và Tem kiểm định cho chủ xe cơ giới hoặc người đưa xe đến
kiểm định theo quy định tại Thông tư số
    
     16/2021/TT-BGTVT
    
    ngày 12/8/2021 của Bộ trưởng Bộ Giao thông vận tải quy định về 

In [12]:
results = query("Giá lập hồ sơ với xe cơ giới?")

for i in results[0]:
    print(f"id: {i.id}, distance: {i.distance}")
print()

for i in results[0]:
    print(i.entity.get("content_text"))

id: 452820403934781324, distance: 0.032786883413791656
id: 452820403934781323, distance: 0.032258063554763794
id: 452820403934775903, distance: 0.016393441706895828

2.
    
     Lập hồ sơ phương tiện đối
với xe cơ giới được miễn kiểm định lần đầu
    
    là việc đơn vị đăng kiểm kiểm
tra, đánh giá hồ sơ phương tiện để lập hồ sơ phương tiện, cấp Giấy chứng nhận
kiểm định, Tem kiểm định đối với xe cơ giới được miễn kiểm định lần đầu theo
quy định tại Thông tư số
    
     16/2021/TT-BGTVT
    
    ngày 12/8/2021 của Bộ trưởng Bộ Giao thông vận tải quy định về kiểm định an
toàn kỹ thuật và bảo vệ môi trường phương tiện giao thông cơ giới đường bộ (được
sửa đổi, bổ sung bởi Thông tư số
    
     02/2023/TT-BGTVT
    
    ngày 21/3/2023, Thông tư số
    
     08/2023/TT-BGTVT
    
    ngày 02/06/2023).
   

    3.
    
     In lại Giấy chứng nhận kiểm
định và Tem kiểm định đối với xe cơ giới
    
    là việc đơn vị đăng kiểm in lại Giấy
chứng nhận kiểm định và Tem kiểm định cho chủ xe cơ 